In [1]:
import sys, os
sys.path.append('../modules')
from nsw_classifier import NSWClassifier

In [4]:
C = NSWClassifier()
C.build_navigable_graph([([1, 2], 3), ([1.2, 2], 3), ((3, 4), 5), ((4, 4), 5), ((6, 4), 5)])
C.cut

Classifer process. Data dimensionality detected is 2. regularity = 6


{(2, 0), (2, 1), (3, 0), (3, 1), (4, 1)}

In [18]:
c = C.classify_by_path([1.1, 2.2], attempts=17)
fc = C.classify_fuzzy_by_path([1.1, 2.2], attempts=17)

print(f"Class = {c}, Fuzzy Classes = {fc}")

Class = 5, Fuzzy Classes = {(3, 0.4117647058823529), (5, 0.5882352941176471)}
